In [111]:
import pandas as pd
import numpy as np

In [112]:
df=pd.read_csv("sales.csv")

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   400 non-null    int64 
 1   CompPrice    400 non-null    int64 
 2   Income       400 non-null    int64 
 3   Advertising  400 non-null    int64 
 4   Population   400 non-null    int64 
 5   Price        400 non-null    int64 
 6   ShelveLoc    400 non-null    object
 7   Age          400 non-null    int64 
 8   Education    400 non-null    int64 
 9   Urban        400 non-null    object
 10  US           400 non-null    object
 11  high         400 non-null    object
dtypes: int64(8), object(4)
memory usage: 37.6+ KB


In [114]:
df.head()

,Unnamed: 0,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,high
0,1,138,73,11,276,120,Bad,42,17,Yes,Yes,yes
1,2,111,48,16,260,83,Good,65,10,Yes,Yes,yes
2,3,113,35,10,269,80,Medium,59,12,Yes,Yes,yes
3,4,117,100,4,466,97,Medium,55,14,Yes,Yes,no
4,5,141,64,3,340,128,Bad,38,13,Yes,No,no


In [115]:
from sklearn.preprocessing import LabelEncoder

In [116]:
le=LabelEncoder()

In [117]:
df["ShelveLoc"]=le.fit_transform(df["ShelveLoc"])
df["Urban"]=le.fit_transform(df["Urban"])
df["US"]=le.fit_transform(df["US"])

In [118]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [119]:
df.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,high
0,138,73,11,276,120,0,42,17,1,1,yes
1,111,48,16,260,83,1,65,10,1,1,yes
2,113,35,10,269,80,2,59,12,1,1,yes
3,117,100,4,466,97,2,55,14,1,1,no
4,141,64,3,340,128,0,38,13,1,0,no


In [120]:
x=df.drop("high",axis=1)


In [121]:
x

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,138,73,11,276,120,0,42,17,1,1
1,111,48,16,260,83,1,65,10,1,1
2,113,35,10,269,80,2,59,12,1,1
3,117,100,4,466,97,2,55,14,1,1
4,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1
396,139,23,3,37,120,2,55,11,0,1
397,162,26,12,368,159,2,40,18,1,1
398,100,79,7,284,95,0,50,12,1,1


In [122]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
new=ss.fit_transform(x)
new=pd.DataFrame(new,columns=x.columns)

In [123]:
X=new[new.columns]
y=df["high"]

In [124]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [125]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [126]:
bagging=BaggingClassifier(estimator=DecisionTreeClassifier(criterion="gini"),n_estimators=100,random_state=42,max_samples=0.6,max_features=4)

In [127]:
bag=bagging.fit(x_train,y_train)

In [128]:
y_train_pred=bag.predict(x_train)
y_test_pred=bag.predict(x_test)

In [129]:
from sklearn.metrics import accuracy_score
print("Train accuracy:",accuracy_score(y_train,y_train_pred))
print("Test accuracy:",accuracy_score(y_test,y_test_pred))

Train accuracy: 1.0
Test accuracy: 0.725


In [130]:
from sklearn.ensemble import RandomForestClassifier

In [131]:
randomforest=RandomForestClassifier(n_estimators=100,random_state=42,max_samples=0.6,max_features=4)

In [132]:
rf=randomforest.fit(x_train,y_train)

In [133]:
y_train_pred=rf.predict(x_train)
y_test_pred=rf.predict(x_test)

In [134]:
print("Train accuracy:",accuracy_score(y_train,y_train_pred))
print("Test accuracy:",accuracy_score(y_test,y_test_pred))

Train accuracy: 0.9892857142857143
Test accuracy: 0.7833333333333333


In [135]:
from sklearn.model_selection import GridSearchCV

In [136]:
param_grid={"n_estimators":[50,100,200],"max_features":[0.6,0,7,0,8],"max_samples":[0.6,0.7,0.8]}

In [137]:
rf1=RandomForestClassifier(random_state=42)

In [138]:
grid_search=GridSearchCV(estimator=rf1,param_grid=param_grid,cv=5,scoring="accuracy")

In [139]:
search=grid_search.fit(x_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
90 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/

In [140]:
search.best_params_

{'max_features': 8, 'max_samples': 0.8, 'n_estimators': 50}

In [141]:
search.best_score_

np.float64(0.8107142857142857)

In [143]:
y_test_pred_best=search.predict(x_test)

In [145]:
print("Best Test accuracy:",accuracy_score(y_test,y_test_pred_best))

Best Test accuracy: 0.7416666666666667
